## Highlights

* Groupby sem ordenação economizando processamento
* Groupby nas colunas
* Groupby criando-se uma função que cria o mapping
* Substituir vários groupbys iguais por groupeds deixa o código mais limpinho
* Grupos criados pelo mapping tem nome e cara
* Podemos fazer um agrupamento utilizando o parâmetro `as_index=False` que ele não mandará as chaves para o index! Ótimo!
* What is the meaning of: Cython-optimized aggregation functions?
* Resample: You never use

## 1. Splitting an object into groups

In [1]:
import pandas as pd

import numpy as np

In [17]:
df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
                           'foo', 'bar', 'foo', 'foo'],
                    'B' : ['one', 'one', 'two', 'three',
                           'two', 'two', 'one', 'three'],
                    'C' : np.random.randn(8),
                    'D' : [100]*8})

In [27]:
df

,A,B,C,D
0,foo,one,0.264834,100
1,bar,one,-0.803832,100
2,foo,two,-0.536437,100
3,bar,three,-0.691424,100
4,foo,two,0.745520,100
5,bar,two,-1.166605,100
6,foo,one,-1.471297,100
7,foo,three,0.176468,100


Split by index (rows)

In [25]:
df.groupby('A').C.sum()

A
bar   -2.661861
foo   -0.820912
Name: C, dtype: float64

Split by columns (not understand very well when I would use this)

In [18]:
def get_letter_type(letter):
    if letter.lower() in 'aeiou':
        return 'vowel'
    else:
        return 'consonant'

In [19]:
grouped = df.groupby(get_letter_type, axis=1)

In [21]:
grouped.D.sum()

consonant    300
vowel        100
Name: D, dtype: int64

In [23]:
grouped.sum()

,consonant,vowel
0,100.264834,foo
1,99.196168,bar
2,99.463563,foo
3,99.308576,bar
4,100.745520,foo
5,98.833395,bar
6,98.528703,foo
7,100.176468,foo


In [28]:
lst = [1, 2, 3, 1, 2, 3]

In [30]:
s = pd.Series([1, 2, 3, 10, 20, 30], lst)

In [39]:
grouped = s.groupby(level=0, sort=False) # The last argument is a potential speedups

Note: This is a valid `mapping`, but no spliting occurs here yet.

In [36]:
grouped

Now we have a `split`

In [35]:
grouped.first() # grouped.last() or sum()

1    1
2    2
3    3
dtype: int64

In [40]:
df3 = pd.DataFrame({'X' : ['A', 'B', 'A', 'B'], 'Y' : [1, 4, 3, 2]})

In [48]:
df3

,X,Y
0,A,1
1,B,4
2,A,3
3,B,2


In [43]:
# [?] When I pass the label that i want to map, we have a straighforward mapping?

df3.groupby(['X']).get_group('A') # its like a filter?

,X,Y
0,A,1
2,A,3


In [44]:
df3.groupby(['X']).groups

{'A': Int64Index([0, 2], dtype='int64'),
 'B': Int64Index([1, 3], dtype='int64')}

In [45]:
df3.groupby(get_letter_type, axis=1).groups

{'consonant': Index(['X', 'Y'], dtype='object')}

In [47]:
df3.groupby(get_letter_type, axis=1).sum()

,consonant
0,1
1,4
2,3
3,2


GroupBy with MultiIndex

In [49]:
arrays = [['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
   ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']] 

In [51]:
index = pd.MultiIndex.from_arrays(arrays, names=['first', 'second'])

In [53]:
s = pd.Series(np.random.randn(8), index=index)

In [57]:
s.groupby(level=[0, 'second']).sum()

first  second
bar    one       0.473368
       two       0.530753
baz    one      -1.067948
       two      -0.925217
foo    one       0.220574
       two       1.037492
qux    one       2.171332
       two       0.332948
dtype: float64

In [58]:
s.sum(level='first')

first
bar    1.004120
baz   -1.993165
foo    1.258066
qux    2.504280
dtype: float64

Grouping DataFrame with Index Levels and Columns¶

In [60]:
arrays = [['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
        ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']]

In [61]:
index = pd.MultiIndex.from_arrays(arrays, names=['first', 'second'])

In [62]:
df = pd.DataFrame({'A': [1, 1, 1, 1, 2, 2, 3, 3],
                   'B': np.arange(8)},
                   index=index)

In [71]:
df.groupby(['first', 'A']).sum()

,,B
first,A,
bar,1,1
baz,1,5
foo,2,9
qux,3,13


In [72]:
df.groupby([pd.Grouper(level='first'), 'A']).sum()

,,B
first,A,
bar,1,1
baz,1,5
foo,2,9
qux,3,13


## 2. Iterating through groups

In [74]:
df

A  B
first second      
bar   one     1  0
      two     1  1
baz   one     1  2
      two     1  3
foo   one     2  4
      two     2  5
qux   one     3  6
      two     3  7

In [79]:
for mapping, group in df.groupby(['first', 'A']):
    print(mapping)
    print(group)
    print("\n\n")

('bar', 1)
              A  B
first second      
bar   one     1  0
      two     1  1



('baz', 1)
              A  B
first second      
baz   one     1  2
      two     1  3



('foo', 2)
              A  B
first second      
foo   one     2  4
      two     2  5



('qux', 3)
              A  B
first second      
qux   one     3  6
      two     3  7





## 3. Selecting a group

Highlights: Podemos filtrar um determinado grupo de maneira mais elegante através do `grouped.get_group()`, tipo 'VENDA', 'APARTAMENTO'

## 4. Aggregation

In [107]:
grouped = df.groupby('first')

grouped.agg({
    'B': np.sum,
    'A': [np.sum] #, lambda ser: ser.nth(n=0)] # Get first element of each group - Não estou conseguindo! [?]
})

,B,A
,sum,sum
first,,
bar,1,2
baz,5,2
foo,9,4
qux,13,6


## 5. Transformation 

Classical example: zscore

In [115]:
index = pd.date_range('10/1/1999', periods=1100)

In [116]:
ts = pd.Series(np.random.normal(0.5, 2, 1100), index)

In [117]:
ts1 = ts.rolling(window=100,min_periods=100).mean().dropna()

In [124]:
ts.head()

1999-10-01    1.385763
1999-10-02    5.681738
1999-10-03    3.880985
1999-10-04    2.371951
1999-10-05    0.566938
Freq: D, dtype: float64

In [123]:
ts1.head()

2000-01-08    0.670902
2000-01-09    0.641459
2000-01-10    0.583607
2000-01-11    0.541563
2000-01-12    0.546638
Freq: D, dtype: float64

In [126]:
zscore = lambda x: (x - x.mean()) / x.std()

In [127]:
transformed = ts.groupby(lambda x: x.year).transform(zscore)

In [128]:
transformed

1999-10-01    0.330736
1999-10-02    2.386934
1999-10-03    1.525033
1999-10-04    0.802759
1999-10-05   -0.061182
1999-10-06    0.987805
1999-10-07    0.267784
1999-10-08    1.612330
1999-10-09   -0.481006
1999-10-10    1.477450
1999-10-11   -1.662083
1999-10-12   -0.233405
1999-10-13   -1.131126
1999-10-14    0.205462
1999-10-15    0.075994
1999-10-16   -0.003643
1999-10-17    1.398375
1999-10-18   -0.460055
1999-10-19   -1.107146
1999-10-20    0.931937
1999-10-21    0.481449
1999-10-22   -0.187519
1999-10-23    0.930567
1999-10-24   -2.149829
1999-10-25    0.636612
1999-10-26    1.524306
1999-10-27   -0.045555
1999-10-28   -0.511217
1999-10-29   -0.340119
1999-10-30   -0.469680
                ...   
2002-09-05    0.843670
2002-09-06    1.333133
2002-09-07   -0.526954
2002-09-08    1.073941
2002-09-09   -0.797679
2002-09-10    0.288842
2002-09-11    0.679092
2002-09-12   -0.467750
2002-09-13    0.088174
2002-09-14   -1.291713
2002-09-15    0.795335
2002-09-16    1.256960
2002-09-17 

In [129]:
# Parei no gráfico